In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

In [2]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [3]:
!ls bin

barcode_rank_functions.R  scrna_plot_qc_metrics.R  scrna_qc_metrics.py


In [4]:
# !chmod +x ./bin/scrna_qc_metrics.py
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_rna/bin'

In [5]:
!chmod +x {os.path.join(bin_fld, "scrna_qc_metrics.py")}
!chmod +x {os.path.join(bin_fld, "scrna_plot_qc_metrics.R")}
!chmod +x {os.path.join(bin_fld, "barcode_rank_functions.R")}

In [6]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [7]:
import os
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'
!echo $TOWER_ACCESS_TOKEN

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz


In [8]:
!echo $ENV_SYNAPSE_TOKEN

In [9]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [10]:
!nextflow secrets get SYNAPSE

null


In [11]:
# TODO: when you run at on your own machine, please change this value of nf_data
nf_data_base_path = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/'

# Print the value to the console
print(f"The nf_data_base_path is: {nf_data_base_path}")

# Write the configuration file
with open("nextflow_vm.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nextflow pipeline for fg single cell' 
  mainScript = 'nf_wf_fg_singlecell_rna.nf'
}}

profiles {{
  conda_profile {{
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
  }}

  docker_local {{
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['TOWER_ACCESS_TOKEN']
    executor = 'local'
  }}

  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/RNA'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}
}}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.ORGANISM = 'human'
params.FASTQS_SPEC_CH = "{nf_data_base_path}nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv"
params.GENOME_FASTA = 'http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF = 'http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()
params.SYNAPSE_CONFIG = "{nf_data_base_path}nf_wf_fg_singlecell/synapse.config"
params.SYNAPSE_IDS = "{nf_data_base_path}nf_wf_fg_singlecell/synapse_ids.csv"
params.SUBPOOL = 'no_subpool'
params.UMI_CUTOFF = 2
params.GENE_CUTOFF = 3
params.SC_RNA_QC_SCRIPT = 'scrna_qc_metrics.py'
params.SC_RNA_QC_PLOT_SCRIPT = 'scrna_plot_qc_metrics.R'
params.SC_RNA_QC_PLOT_HELPER_SCRIPT = 'barcode_rank_functions.R'
params.RNA_DEBUG_QC = "{nf_data_base_path}nf_workflow/workflow_fg_single_cell_rna/sc_rna_qc_calculate.tsv"
""")


The nf_data_base_path is: /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/


In [12]:
# !rm -rf /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/work
# !gsutil -m rm -r gs://eila-nextflow-bucket/work/RNA


In [13]:

!nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume
# -resume 
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile  -with-tower

# docker_local



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile docker_profile

N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_rna.nf` [intergalactic_hypatia] DSL2 - revision: 72f4ca21b5
Dec 18, 2023 3:23:49 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
null
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv
[-        ] process > run_seqspec_modify_rna -
[-        ] process > run_seqspec_print      -
after run_seqspec_modify_rna
after run_seqspec_print

[-        ] process > run_seqspec_modify_rna     -
[-        ] process > run_seqspec_print          -
[-        ] process > run_seqspec_index_rna_kb   -
[-        ] process > run_kb_ref                 -
[-        ] process > run_kb_count               -
[-        ] process > scrna_calculate_qc_metrics -
[-        ] process > scrna_plot_qc_metrics_prcs -
after run_seqspec_modify_rna
after run_seqspec_print
after run_seqspec_index_rna_kb
technology is $technology
after genome_fasta_ch
after

In [14]:
!cat .nextflow.log 

Dec-18 03:23:45.367 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume
Dec-18 03:23:45.785 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Dec-18 03:23:45.824 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Dec-18 03:23:45.842 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Dec-18 03:23:45.844 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Dec-18 03:23:45.850 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Dec-18 03:23:45.868 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Dec-18 03:23:45.902 [main] DEBUG nextflow.config.C

In [15]:
import datetime
now = datetime.datetime.now()
print(now)

2023-12-18 03:30:37.439475


In [16]:

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume 
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume 886dc4ef-ff60-46d0-aecb-8f1ab56296a3

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume





# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile google_batch_profile -resume

In [17]:
# !nextflow log
# use the session id with the resume

In [18]:
# !cat .nextflow.log 

In [19]:
import datetime
now = datetime.datetime.now()
print(now)

2023-12-18 03:30:37.471613
